In [1]:
import sima_motion_correction
reload(sima_motion_correction)
import glob
import multiprocessing as mp

In [3]:
%%time

fdir = 'C:\\2pData\\Sean\\'
max_disp = [30, 50] 
save_displacement = False

# gather all tif or h5 files in root directory
fpaths = []
types = ['*.tif', '*.tiff', '*.h5']
for type in types:
    fpaths.extend(glob.glob(fdir + type))
print(str(len(fpaths)) + ' files to analyze')

# perform parallel processing
num_processes = min(mp.cpu_count(), len(fpaths))
print( 'Total CPU cores for parallel processing: ' + str(num_processes) )

pool = mp.Pool(processes = num_processes)
pool.map(sima_motion_correction.unpack, [(file, max_disp) for file in fpaths])
pool.close()
pool.join()

2 files to analyze
Total CPU cores for parallel processing: 2
Wall time: 4min 1s
